# Web Scraping with Beautiful Soup: Solutions

In [1]:
# Import required libraries
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

In [13]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp')
# Read the content of the server’s response
src = req.text
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')

In [20]:
print(soup)

<!DOCTYPE html>
<html lang="en">
<head id="Head1">
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta charset="utf-8"/>
<!-- Meta Description -->
<meta content="Welcome to the official government website of the Illinois General Assembly" name="description"/>
<meta content="Legislative Information System" name="contactName"/>
<meta content="LIS Staff Services" name="contactOrganization"/>
<meta content="705 Stratton Office Building" name="contactStreetAddress1"/>
<meta content="Springfield" name="contactCity"/>
<meta content="62706" name="contactZipcode"/>
<meta content="webmaster@ilga.gov" name="contactNetworkAddress"/>
<meta content="217-782-3944" name="contactPhoneNumber"/>
<meta content="217-524-6059" name="contactFaxNumber"/>
<meta content="State Of Illinois" name="originatorJurisdiction"/>
<meta content="Illin

## Challenge: Find All

Use Beautiful Soup to find all the `a` elements with class `mainmenu`.

In [19]:
soup.select("a.mainmenu")

[]

## Challenge: Extract Specific Attributes

Extract all `href` attributes for each `mainmenu` URL.

In [4]:
[link['href'] for link in soup.select("a.mainmenu")]

[]

## Challenge: Get `href` elements pointing to members' bills

The code above retrieves information on:  

- the senator's name,
- their district number,
- and their party.

We now want to retrieve the URL for each senator's list of bills. Each URL will follow a specific format.

The format for the list of bills for a given senator is:

`http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=[MEMBER_ID]&Primary=True`

to get something like:

`http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True`

in which `MEMBER_ID=1911`.

You should be able to see that, unfortunately, `MEMBER_ID` is not currently something pulled out in our scraping code.

Your initial task is to modify the code above so that we also **retrieve the full URL which points to the corresponding page of primary-sponsored bills**, for each member, and return it along with their name, district, and party.

Tips:

* To do this, you will want to get the appropriate anchor element (`<a>`) in each legislator's row of the table. You can again use the `.select()` method on the `row` object in the loop to do this — similar to the command that finds all of the `td.detail` cells in the row. Remember that we only want the link to the legislator's bills, not the committees or the legislator's profile page.
* The anchor elements' HTML will look like `<a href="/senate/Senator.asp/...">Bills</a>`. The string in the `href` attribute contains the **relative** link we are after. You can access an attribute of a BeatifulSoup `Tag` object the same way you access a Python dictionary: `anchor['attributeName']`. See the <a href="http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag">documentation</a> for more details.
* There are a _lot_ of different ways to use BeautifulSoup to get things done. whatever you need to do to pull the `href` out is fine.

The code has been partially filled out for you. Fill it in where it says `#YOUR CODE HERE`. Save the path into an object called `full_path`.

In [5]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")
# Create empty list to store our data
members = []

# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')
# Get rid of junk rows
rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail')
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]

    # YOUR CODE HERE
    # Extract href
    href = row.select('a')[1]['href']
    # Create full path
    full_path = "http://www.ilga.gov/senate/" + href + "&Primary=True"

    # Store in a tuple
    senator = (name, district, party, full_path)
    # Append to list
    members.append(senator)

In [6]:
members[:5]

[]

## Challenge: Modularize Your Code

Turn the code above into a function that accepts a URL, scrapes the URL for its senators, and returns a list of tuples containing information about each senator.

In [7]:
def get_members(url):
    # Make a GET request
    req = requests.get(url)
    # Read the content of the server’s response
    src = req.text
    # Soup it
    soup = BeautifulSoup(src, "lxml")
    # Create empty list to store our data
    members = []

    # Returns every ‘tr tr tr’ css selector in the page
    rows = soup.select('tr tr tr')
    # Get rid of junk rows
    rows = [row for row in rows if row.select('td.detail')]

    # Loop through all rows
    for row in rows:
        # Select only those 'td' tags with class 'detail'
        detail_cells = row.select('td.detail')
        # Keep only the text in each of those cells
        row_data = [cell.text for cell in detail_cells]
        # Collect information
        name = row_data[0]
        district = int(row_data[3])
        party = row_data[4]

        # YOUR CODE HERE
        # Extract href
        href = row.select('a')[1]['href']
        # Create full path
        full_path = "http://www.ilga.gov/senate/" + href + "&Primary=True"

        # Store in a tuple
        senator = (name, district, party, full_path)
        # Append to list
        members.append(senator)
    return(members)

In [8]:
# Test your code!
url = 'http://www.ilga.gov/senate/default.asp?GA=98'
senate_members = get_members(url)
len(senate_members)

0

## Take-home Challenge: Writing a Scraper Function

We want to scrape the webpages corresponding to bills sponsored by each bills.

Write a function called `get_bills(url)` to parse a given bills URL. This will involve:

  - requesting the URL using the <a href="http://docs.python-requests.org/en/latest/">`requests`</a> library
  - using the features of the `BeautifulSoup` library to find all of the `<td>` elements with the class `billlist`
  - return a _list_ of tuples, each with:
      - description (2nd column)
      - chamber (S or H) (3rd column)
      - the last action (4th column)
      - the last action date (5th column)
      
This function has been partially completed. Fill in the rest.

In [9]:
def get_bills(url):
    src = requests.get(url).text
    soup = BeautifulSoup(src)
    rows = soup.select('tr tr tr')
    bills = []
    # Iterate over rows
    for row in rows:
        # Grab all bill list cells
        cells = row.select('td.billlist')
        # Keep in mind the name of the senator is not a billlist class!
        if len(cells) == 5:
            row_text = [cell.text for cell in cells]
            # Extract info from row text
            bill_id = row_text[0]
            description = row_text[1]
            chamber = row_text[2]
            last_action = row_text[3]
            last_action_date = row_text[4]
            # Consolidate bill info
            bill = (bill_id, description, chamber, last_action, last_action_date)
            bills.append(bill)
    return bills

In [10]:
test_url = senate_members[0][3]
get_bills(test_url)[0:5]

IndexError: list index out of range

### Scrape All Bills

Finally, create a dictionary `bills_dict` which maps a district number (the key) onto a list of bills (the value) coming from that district. You can do this by looping over all of the senate members in `members_dict` and calling `get_bills()` for each of their associated bill URLs.

**NOTE:** please call the function `time.sleep(1)` for each iteration of the loop, so that we don't destroy the state's web site.

In [11]:
bills_dict = {}
for member in senate_members[:5]:
    bills_dict[member[1]] = get_bills(member[3])
    time.sleep(1)

In [12]:
len(bills_dict[52])

KeyError: 52

# Versión Corregida

El workbook correspondiente busca realizar un scrapping a la siguiente [página gubernamental](http://www.ilga.gov/senate/default.asp) en la cual se tiene a los senadores de Estados Unidos, adicionalmente vamos a explicar los errores y que se realiza en cada paso y vamos a presentar una versión corregida del mismo proyecto.

In [ ]:
# Import required libraries
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp')
# Read the content of the server’s response
src = req.text
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')

En este primer bloque lo que se hace realizar la importación de las siguientes librerias:

**BeautifulSoup** → librería de bs4 que sirve para parsear HTML/XML y poder navegarlo como un árbol (en lugar de tener solo texto plano).

**datetime** → módulo estándar de Python para trabajar con fechas y horas. Aún no se usa en tu snippet, pero podría servir luego para registrar cuándo se hace el scraping.

**requests** → librería para hacer solicitudes HTTP (GET, POST, etc.). Aquí la usarás para descargar la página web.

**time** → módulo estándar para manejar pausas y tiempos (por ejemplo, time.sleep(2) pausa 2 segundos entre requests). Es útil en scraping para no sobrecargar un servidor.

En el segundo bloque, utilizando ***requests.get(url)*** se hace una solicitud HTTP tipo GET al servidor de la página del Senado de Illinois.

El resultado (req) es un objeto Response que contiene:

1.   Código de estado (200 si fue exitoso).
2.   Encabezados (headers).
3.   Contenido de la respuesta (el HTML completo).


***req.text*** devuelve el contenido de la página como un string.

Ahora ***src*** contiene todo el HTML crudo de la página.

El ***soup = BeautifulSoup(src, 'lxml')*** se convierte ese HTML en una estructura de árbol DOM, el parser 'lxml' es rápido y maneja mejor HTML “imperfecto”, ***soup*** ya no es un texto plano: es un objeto en el que puedes buscar, recorrer y extraer etiquetas.


In [22]:
soup.select("a.mainmenu")
[link['href'] for link in soup.select("a.mainmenu")]

[]

La línea soup.select("a.mainmenu") usa BeautifulSoup para buscar en el árbol HTML todos los elementos de tipo `<a>` (enlaces) que tengan la clase CSS "mainmenu". El método .select() permite usar selectores de CSS, igual que en desarrollo web (por ejemplo, a.mainmenu significa “todos los `<a>` con class="mainmenu"`”). El resultado es una lista de enlaces, cada uno representado como un objeto BeautifulSoup, que luego puedes recorrer para extraer información como el texto visible (link.text) o la dirección del enlace (link['href']).

**Error**: Al momento de ejecutar estas dos líneas se obtiene una lista vacía por lo que se procedió a analizar la respuesta que se obtuvo de la variable ***soup***, y obtuvo lo siguiente:

In [ ]:
#   <div class="col pb-3 flex-wrap member-card-container justify-content-center d-md-flex d-none" data-name="craig wilcox" style="display: flex;">
#   <div class="flex-column justify-content-between">
#   <div class="member-card mb-4" onclick="goToURL('Members/Details/3336')" style="background-image: url('https://cdn.ilga.gov/assets/img/members/%7BE397F861-68AA-4CC2-9A23-50B9A8E13031%7D.jpg');">
#   <div class="member-overlay">
#   <h5 class="card-title"><a class="notranslate" href="/Senate/Members/Details/3336">Craig Wilcox</a> (R)</h5>
#   <p class="card-text">
#                                            Senator
#                                            <br/>32nd District
#                                        </p>
#   </div>
#   </div>
#   </div>
#   </div>

In [21]:
# ¿Cuántas tarjetas de miembros encontramos?
cards = soup.select("div.member-card-container")
print("Cards found:", len(cards))

# Revisar HTML interno de una tarjeta para confirmar la estructura

if cards:
    print(cards[0].prettify()[:800])  # preview first 800 chars of the first card

Cards found: 120
<div class="col pb-3 flex-wrap member-card-container justify-content-center d-md-flex d-none" data-name="neil anderson" style="display: flex;">
 <div class="flex-column justify-content-between">
  <div class="member-card mb-4" onclick="goToURL('Members/Details/3312')" style="background-image: url('https://cdn.ilga.gov/assets/img/members/%7B90CDA259-1DEA-4D18-AE97-30051E03D154%7D.jpg');">
   <div class="member-overlay">
    <h5 class="card-title">
     <a class="notranslate" href="/Senate/Members/Details/3312">
      Neil Anderson
     </a>
     (R)
    </h5>
    <p class="card-text">
     Republican Caucus Chair
     <br/>
     47th District
    </p>
   </div>
  </div>
 </div>
</div>



El código toma el contenido HTML de la página y lo transforma en una estructura que Python puede entender gracias a BeautifulSoup con el analizador lxml. Después busca todos los bloques ***div*** que tengan la clase "member-card-container", que son las tarjetas donde aparece la información de cada miembro del Senado, e imprime cuántas de esas tarjetas se encontraron. Por último, si al menos existe una, muestra de manera ordenada los primeros 800 caracteres del HTML de la primera tarjeta, para verificar que la estructura obtenida es la correcta sin necesidad de desplegar todo el código completo.

Como se puede observar el problema principal reside que el código anterior realiza una busqueda en etiquetas que no contienen información, por lo que se procedió a analizar la respuesta y poder obtener información de las etiquetas con información útil.

***Error en el resto del código:*** Anteriormente, el ejercicio planteaba usar la estructura de enlaces con la clase a.mainmenu para obtener las direcciones (href) de las páginas de cada senador y, a partir de ahí, construir la URL con la lista de proyectos de ley. Sin embargo, la página del Senado de Illinois cambió su estructura: los enlaces ya no se encuentran en un menú con esa clase, sino dentro de bloques más complejos llamados div.member-card-container, donde la información del senador (nombre, partido, distrito y enlace a su perfil) está organizada de manera distinta. Esto significa que el código original basado en a.mainmenu no devuelve resultados, porque esa clase ya no existe en el HTML actual. Por esa razón fue necesario modificar el scraping para adaptarse a la nueva estructura, extrayendo los enlaces desde el atributo href de las etiquetas `<a>` dentro de cada tarjeta de miembro, y a partir de allí construir la URL completa que apunta a la página de proyectos de ley de cada senador junto con sus demás datos.




In [23]:
from urllib.parse import urljoin
import re

In [24]:
BASE = "https://www.ilga.gov"

def parse_member_card(card):
    """
    Extrae: name, party, href absoluto, position y district desde un 'div.member-card-container'.
    """
    # Título con <a class="notranslate">Nombre</a> (D)/(R)
    title = card.select_one("h5.card-title")
    if not title or not title.a:
        return None  # estructura inesperada

    name = title.a.get_text(strip=True)

    # El partido suele estar como texto inmediatamente después del <a>, p. ej. "(D)"
    party_raw = (title.a.next_sibling or "").strip()
    m = re.search(r"\(([A-Z])\)", party_raw)  # captura "D" o "R"
    party = m.group(1) if m else ""

    # href al perfil (relativo → absoluto)
    href_rel = title.a.get("href", "").strip()
    href = urljoin(BASE, href_rel)

    # Cargo + distrito en <p class="card-text"> con un <br/>
    info_tag = card.select_one("p.card-text")
    info = info_tag.get_text(" ", strip=True) if info_tag else ""   # p.ej. "Senator 27th District"

    # Separar position y district si es posible
    position, district = info, ""
    m2 = re.search(r"(.+?)\s+(\d+(?:st|nd|rd|th)\s+District)$", info)
    if m2:
        position = m2.group(1).strip()
        district = m2.group(2).strip()

    return {
        "name": name,
        "party": party,            # "D" o "R" si se detectó
        "party_raw": party_raw,    # "(D)" literal
        "url": href,               # perfil absoluto
        "position": position,      # "Senator"
        "district": district,      # "27th District"
        "position_district": info  # original por si acaso
    }

In [25]:
# Probar con la primera tarjeta
sample = parse_member_card(cards[0])
sample

{'name': 'Neil Anderson',
 'party': 'R',
 'party_raw': '(R)',
 'url': 'https://www.ilga.gov/Senate/Members/Details/3312',
 'position': 'Republican Caucus Chair',
 'district': '47th District',
 'position_district': 'Republican Caucus Chair 47th District'}

In [27]:
url = "https://www.ilga.gov/Senate/Members/Details/3312"  # ejemplo: Mark L. Walker
headers = {"User-Agent": "Mozilla/5.0"}

resp = requests.get(url, headers=headers)
resp.raise_for_status()

soup_detail = BeautifulSoup(resp.text, "lxml")

print(soup_detail)  # título de la página, para validar

<!DOCTYPE html>
<html lang="en">
<head id="Head1">
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta charset="utf-8"/>
<!-- Meta Description -->
<meta content="Welcome to the official government website of the Illinois General Assembly" name="description"/>
<meta content="Legislative Information System" name="contactName"/>
<meta content="LIS Staff Services" name="contactOrganization"/>
<meta content="705 Stratton Office Building" name="contactStreetAddress1"/>
<meta content="Springfield" name="contactCity"/>
<meta content="62706" name="contactZipcode"/>
<meta content="webmaster@ilga.gov" name="contactNetworkAddress"/>
<meta content="217-782-3944" name="contactPhoneNumber"/>
<meta content="217-524-6059" name="contactFaxNumber"/>
<meta content="State Of Illinois" name="originatorJurisdiction"/>
<meta content="Illin

In [28]:
def fetch_soup(url: str) -> BeautifulSoup:
    full = url if url.startswith("http") else urljoin(BASE, url)
    r = requests.get(full, headers=headers, timeout=30)
    r.raise_for_status()
    return BeautifulSoup(r.text, "lxml")

def extract_period_and_bio(soup: BeautifulSoup) -> dict:
    # Periodo (ej. "2015 - Present") en <p class="text-muted mb-1">
    period = ""
    tag_period = soup.select_one("p.text-muted.mb-1")
    if tag_period:
        period = tag_period.get_text(" ", strip=True)

    # Biografía: encontrar el h3 cuyo texto sea "Biography" (case-insensitive)
    bio_text = ""
    h3_bio = None
    for h in soup.select("h3"):
        if h.get_text(strip=True).lower() == "biography":
            h3_bio = h
            break

    if h3_bio:
        # Tomar los párrafos hermanos siguientes hasta el próximo título/bloque fuerte
        parts = []
        for sib in h3_bio.find_all_next():
            # Cortar si aparece otro encabezado de sección
            if sib.name in {"h2", "h3"} and sib is not h3_bio:
                break
            if sib.name == "p":
                txt = sib.get_text(" ", strip=True)
                if txt:
                    parts.append(txt)
            # Si el layout mete la bio en un div/lista inmediatamente después
            if len(parts) >= 1 and sib.name in {"div", "section"}:
                # opcional romper si cambia de sección
                pass
        bio_text = " ".join(parts).strip()

    return {"period": period, "biography": bio_text}

In [29]:
def enrich_members_with_profile(members, sleep_sec=0.4):
    enriched = []
    for m in members:
        try:
            soup_detail = fetch_soup(m["url"])
            extra = extract_period_and_bio(soup_detail)
            enriched.append({**m, **extra})
            time.sleep(sleep_sec)  # ser amable con el servidor
        except Exception as e:
            enriched.append({**m, "period": "", "biography": "", "_error": str(e)})
    return enriched

In [31]:
print("len(cards):", len(cards))
print("len(members):", len(members) if 'members' in globals() else "members no existe")

# Si members existe, mira una muestra
if 'members' in globals():
    from itertools import islice
    print(list(islice(members, 3)))

len(cards): 120
len(members): 0
[]


In [32]:
members = []
for i, card in enumerate(cards, start=1):
    data = parse_member_card(card)
    if data:
        members.append(data)
    else:
        print(f"[WARN] Tarjeta #{i} con estructura inesperada")

print("Miembros parseados:", len(members))

Miembros parseados: 120


In [33]:
assert all('url' in m and m['url'] for m in members), "Faltan URLs en members"

In [34]:
enriched_members = enrich_members_with_profile(members)

print("len(enriched_members):", len(enriched_members))
if enriched_members:
    print(enriched_members[0])
else:
    print("enriched_members está vacío; revisa que 'members' tenga elementos y URLs válidas.")

len(enriched_members): 120
{'name': 'Neil Anderson', 'party': 'R', 'party_raw': '(R)', 'url': 'https://www.ilga.gov/Senate/Members/Details/3312', 'position': 'Republican Caucus Chair', 'district': '47th District', 'position_district': 'Republican Caucus Chair 47th District', 'period': 'Republican Caucus Chair', 'biography': 'State Senator Neil Anderson, a professional firefighter and paramedic for the City of Moline, was first elected in 2015 and represents Illinois’ 47th District, spanning 15 counties. Raised in the Quad Cities, he worked in his family’s flooring business and walked on to the University of Nebraska football team, becoming the only walk-on to make the roster.\r\n\r\nAfter college, Senator Anderson pursued his passion for public service, joining the Moline Fire Department in 2006. In 2023, he was appointed Senate Republican Caucus Chair. He lives in Andalusia with his wife, Brandi, and their two children, Steel and Sophia.'}


# Resultado:

El resultado muestra que la función de enriquecimiento logró procesar correctamente la información de 120 miembros del Senado. Al inspeccionar el primer elemento, se observa un diccionario con los datos completos de Neil Anderson: su nombre, el partido al que pertenece (Republicano), el URL de su perfil oficial, así como su cargo y distrito (Republican Caucus Chair, 47th District). Además, se añadió el periodo de servicio, aunque en este caso se mezcló con la posición por cómo estaba estructurado el HTML, y finalmente se extrajo la biografía completa, que describe su trayectoria personal y profesional. Este resultado confirma que el scraping está capturando no solo los datos básicos de las tarjetas de miembros, sino también información enriquecida desde la página de detalle de cada senador.